## Analyze Census Response Rates

In [2]:
import pandas as pd
import altair as alt

# use the most recent csv you pulled
analysis_date = "2020-04-20"

In [6]:
rr = pd.read_csv("data/source/responserates_{0}.csv".format(analysis_date))

# add a column for non-internet responses
rr["CRRNOTINT"] = rr["CRRALL"] - rr["CRRINT"]

rr.head()

,NAME,GEO_ID,DRRALL,DRRINT,CRRALL,CRRINT,RESP_DATE,us,state,county,county subdivision,CRRNOTINT
0,United States,0100000US,0.3,0.2,50.7,44.2,2020-04-20,1.0,NaN,NaN,NaN,6.5
1,Arizona,0400000US04,0.2,0.1,48.4,42.9,2020-04-20,NaN,4.0,NaN,NaN,5.5
2,Connecticut,0400000US09,0.2,0.2,52.7,48.2,2020-04-20,NaN,9.0,NaN,NaN,4.5
3,Arkansas,0400000US05,0.2,0.1,47.0,33.4,2020-04-20,NaN,5.0,NaN,NaN,13.6
4,Colorado,0400000US08,0.2,0.2,54.5,50.9,2020-04-20,NaN,8.0,NaN,NaN,3.6


In [7]:
rr_states = rr[(rr.county.isna()) & (rr["county subdivision"].isna())]

len(rr_states)

53

Chart response rates, including US

In [37]:
bar = alt.Chart(rr_states).mark_bar().encode(
    x="CRRALL",
    y=alt.Y("NAME", sort="-x"),
    color=alt.condition(
        alt.datum.NAME == "United States",
        alt.value('steelblue'),
        alt.value('darkgray')
    )
)
text = alt.Chart(rr_states).mark_text(dx=-8, color="white",align="right").encode(
    x="CRRALL",
    y=alt.Y("NAME", sort="-x"),
    text="CRRALL"
)

bar+text

alt.LayerChart(...)

Filter to just Vermont

In [40]:
vtus = ["United States","Vermont"]
rr_vtus = rr.copy()[rr.NAME.isin(vtus)]
rr_vtus

,NAME,GEO_ID,DRRALL,DRRINT,CRRALL,CRRINT,RESP_DATE,us,state,county,county subdivision,CRRNOTINT
0,United States,0100000US,0.3,0.2,50.7,44.2,2020-04-20,1.0,NaN,NaN,NaN,6.5
35,Vermont,0400000US50,0.2,0.1,41.5,36.4,2020-04-20,NaN,50.0,NaN,NaN,5.1


In [46]:
rr_vtus["int_pct"] = rr_vtus["CRRINT"]/rr_vtus["CRRALL"]

#make a stacked version for visualization
rr_vtus_stack = pd.melt(rr_vtus[["NAME","CRRINT","CRRNOTINT"]], id_vars="NAME")

rr_vtus

,NAME,GEO_ID,DRRALL,DRRINT,CRRALL,CRRINT,RESP_DATE,us,state,county,county subdivision,CRRNOTINT,int_pct
0,United States,0100000US,0.3,0.2,50.7,44.2,2020-04-20,1.0,NaN,NaN,NaN,6.5,0.871795
35,Vermont,0400000US50,0.2,0.1,41.5,36.4,2020-04-20,NaN,50.0,NaN,NaN,5.1,0.877108


In [47]:
alt.Chart(rr_vtus_stack).mark_bar().encode(
    x=alt.X("value"),
    y="NAME",
    color="variable"
)

alt.Chart(...)

In [48]:
alt.Chart(rr_vtus_stack).mark_bar().encode(
    x=alt.X("value",stack="normalize"),
    y="NAME",
    color="variable"
)

alt.Chart(...)

Now let's look at county

In [52]:
rr_county = rr.copy()[(rr.county.notna()) & (rr["county subdivision"].isna())]
rr_county["NAME"] = rr_county["NAME"].str.replace(" County, Vermont","")

rr_county

,NAME,GEO_ID,DRRALL,DRRINT,CRRALL,CRRINT,RESP_DATE,us,state,county,county subdivision,CRRNOTINT
53,Bennington,0500000US50003,0.2,0.1,34.2,28.8,2020-04-20,NaN,50.0,3.0,NaN,5.4
54,Caledonia,0500000US50005,0.2,0.1,35.1,27.2,2020-04-20,NaN,50.0,5.0,NaN,7.9
55,Chittenden,0500000US50007,0.4,0.2,58.2,56.2,2020-04-20,NaN,50.0,7.0,NaN,2.0
56,Essex,0500000US50009,0.1,0.0,19.5,10.1,2020-04-20,NaN,50.0,9.0,NaN,9.4
57,Franklin,0500000US50011,0.3,0.2,45.7,40.2,2020-04-20,NaN,50.0,11.0,NaN,5.5
58,Grand Isle,0500000US50013,0.2,0.1,30.2,29.5,2020-04-20,NaN,50.0,13.0,NaN,0.7
59,Lamoille,0500000US50015,0.2,0.1,36.8,31.5,2020-04-20,NaN,50.0,15.0,NaN,5.3
60,Orange,0500000US50017,0.2,0.1,38.0,29.6,2020-04-20,NaN,50.0,17.0,NaN,8.4
61,Orleans,0500000US50019,0.2,0.1,32.6,17.7,2020-04-20,NaN,50.0,19.0,NaN,14.9
62,Rutland,0500000US50021,0.2,0.1,39.2,29.9,2020-04-20,NaN,50.0,21.0,NaN,9.3


In [53]:
rr_county_pct = rr_county.copy().sort_values("CRRALL", ascending=False)
rr_county_pct["int_pct"] = rr_county_pct["CRRINT"]/rr_county_pct["CRRALL"]
rr_county_pct["noint_pct"] = rr_county_pct["CRRNOTINT"]/rr_county_pct["CRRALL"]
rr_county_pct.drop(["GEO_ID","DRRALL","DRRINT","RESP_DATE","us","state","county","county subdivision"],axis=1,inplace=True)
rr_county_pct.to_clipboard(index=False)
rr_county_pct

,NAME,CRRALL,CRRINT,CRRNOTINT,int_pct,noint_pct
55,Chittenden,58.2,56.2,2.0,0.965636,0.034364
66,Addison,48.5,46.0,2.5,0.948454,0.051546
57,Franklin,45.7,40.2,5.5,0.879650,0.120350
63,Washington,42.8,39.9,2.9,0.932243,0.067757
62,Rutland,39.2,29.9,9.3,0.762755,0.237245
60,Orange,38.0,29.6,8.4,0.778947,0.221053
59,Lamoille,36.8,31.5,5.3,0.855978,0.144022
54,Caledonia,35.1,27.2,7.9,0.774929,0.225071
53,Bennington,34.2,28.8,5.4,0.842105,0.157895
65,Windsor,34.2,30.0,4.2,0.877193,0.122807


In [54]:
rr_county_stack = pd.melt(rr_county[["NAME","CRRINT","CRRNOTINT"]], id_vars="NAME")

rr_county_stack.head()

,NAME,variable,value
0,Bennington,CRRINT,28.8
1,Caledonia,CRRINT,27.2
2,Chittenden,CRRINT,56.2
3,Essex,CRRINT,10.1
4,Franklin,CRRINT,40.2


In [55]:
alt.Chart(rr_county_stack).mark_bar().encode(
    x=alt.X("value"),
    y="NAME",
    color="variable",
    tooltip=list(rr_county_stack.columns)
)

alt.Chart(...)

In [56]:
alt.Chart(rr_county_stack).mark_bar().encode(
    x=alt.X("value", stack="normalize"),
    y="NAME",
    color="variable"
)

alt.Chart(...)